In [19]:
import pandas as pd
from pandas import read_csv
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [15]:
# load a single file as a numpy array
# def load_file(filepath):
#     dataframe = read_csv(filepath, header=None, delim_whitespace=True)
#     return dataframe.values

# load a dataset group, such as train or test
# def load_dataset_group(group, prefix=''):
#     # load input data
#     X = load_file(prefix + group + '/X_'+group+'.txt')
#     # load class output
#     y = load_file(prefix + group + '/y_'+group+'.txt')
#     return X, y

# load the dataset, returns train and test X and y elements
# def load_dataset(prefix=''):
#     # load all train
#     trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
#     print(trainX.shape, trainy.shape)
#     # load all test
#     testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
#     print(testX.shape, testy.shape)
#     # flatten y
#     trainy, testy = trainy[:,0], testy[:,0]
#     print(trainX.shape, trainy.shape, testX.shape, testy.shape)
#     return trainX, trainy, testX, testy

# def load_dataset(prefix=''):
#     # load all train
#     train = load_dataset_group('train', prefix + 'HARDataset/')
#     print(trainX.shape, trainy.shape)
#     # load all test
#     testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
#     print(testX.shape, testy.shape)
#     # flatten y
#     trainy, testy = trainy[:,0], testy[:,0]
#     print(trainX.shape, trainy.shape, testX.shape, testy.shape)
#     return trainX, trainy, testX, testy

In [17]:
# path = '../data/processed/flori5hz.csv'
# values = load_file(path)
# #X, y = load_dataset_group()
# values

In [46]:
train_df = pd.read_csv('../data/processed/flori5hz.csv')
test_df = pd.read_csv('../data/processed/fakhra5hz.csv')
combine = [train_df, test_df]

train_df.describe()

,x,y,z,state
count,3067.000000,3067.000000,3067.000000,3067.000000
mean,0.071598,0.021206,0.988277,0.856537
std,0.016549,0.018619,0.018719,0.350601
min,-0.028320,-0.187500,0.677734,0.000000
25%,0.065430,0.013672,0.982422,1.000000
50%,0.071289,0.020508,0.988281,1.000000
75%,0.077148,0.028320,0.994141,1.000000
max,0.360352,0.310547,1.382812,1.000000


In [47]:
X_train = train_df.drop('state', axis=1)
y_train = train_df['state']
X_test = test_df.drop('state', axis=1)
y_test = test_df['state']

In [38]:
# create a dict of standard models to evaluate {name:object}
def define_models(models=dict()):
    # nonlinear models
    models['knn'] = KNeighborsClassifier(n_neighbors=7)
    models['cart'] = DecisionTreeClassifier()
    models['svm'] = SVC()
    models['bayes'] = GaussianNB()
    # ensemble models
    models['bag'] = BaggingClassifier(n_estimators=100)
    models['rf'] = RandomForestClassifier(n_estimators=100)
    models['et'] = ExtraTreesClassifier(n_estimators=100)
    models['gbm'] = GradientBoostingClassifier(n_estimators=100)
    print('Defined %d models' % len(models))
    return models

In [39]:

# get model list
models = define_models()

Defined 8 models


In [50]:
# evaluate a single model
def evaluate_model(trainX, trainy, testX, testy, model):
    # fit the model
    model.fit(trainX, trainy)
    # make predictions
    yhat = model.predict(testX)
    # evaluate predictions
    accuracy = accuracy_score(testy, yhat)
    return accuracy * 100.0

# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(trainX, trainy, testX, testy, models):
    results = dict()
    for name, model in models.items():
        # evaluate the model
        results[name] = evaluate_model(trainX, trainy, testX, testy, model)
        # show process
        print('>%s: %.3f' % (name, results[name]))
    return results

# print and plot the results
def summarize_results(results, maximize=True):
    # create a list of (name, mean(scores)) tuples
    mean_scores = [(k,v) for k,v in results.items()]
    # sort tuples by mean score
    mean_scores = sorted(mean_scores, key=lambda x: x[1])
    # reverse for descending order (e.g. for accuracy)
    if maximize:
        mean_scores = list(reversed(mean_scores))
    print()
    for name, score in mean_scores:
        print('Name=%s, Score=%.3f' % (name, score))

In [48]:
# evaluate models
results = evaluate_models(X_train, y_train, X_test, y_test, models)

>cart: 91.932
>rf: 92.637


/home/elizabeth/.local/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


>svm: 93.571
>knn: 92.919
>et: 92.778
>bayes: 93.218
>bag: 92.531
>gbm: 92.884


In [51]:
# summarize results
summarize_results(results)


Name=svm, Score=93.571
Name=bayes, Score=93.218
Name=knn, Score=92.919
Name=gbm, Score=92.884
Name=et, Score=92.778
Name=rf, Score=92.637
Name=bag, Score=92.531
Name=cart, Score=91.932
